In [1]:
import os
os.chdir('/high_spd_work/download_twitter_disease_data/')
from prelim_analysis import (
    MakeIter,
    twt_generator,
    twts,
    len_iterable,
    preprocess_twt,
    eng_stopwords,
    cleaned_sentences,
    make_model,
    flatten_list,
    get_word_freq,
    plot_map,
    get_weeknum_from_filename,
    count_related_words_normalized,
    make_heatmap_w2vrelated,
    scikit_pca,
    make_histogram)

%matplotlib inline
%pylab inline



Populating the interactive namespace from numpy and matplotlib


/high_spd_work/download_twitter_disease_data/venv/lib/python3.4/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# preliminary analysis:

In [2]:
# important keys are 'text', 'created_at', 'coordinates'
print("These are the Keys:")
for twt in twts:
    print(twt.keys())
    break

# more than 2 million tweets or >8 Gigs
print("\n Number of english tweets:")
print(len_iterable(twts))

These are the Keys:
dict_keys(['text', 'in_reply_to_user_id_str', 'weeknum', 'coordinates', 'id', 'lang', 'is_quote_status', 'truncated', 'contributors', 'in_reply_to_screen_name', 'in_reply_to_status_id_str', 'filter_level', 'in_reply_to_status_id', 'id_str', 'retweeted', 'in_reply_to_user_id', 'place', 'entities', 'timestamp_ms', 'user', 'created_at', 'favorite_count', 'geo', 'favorited', 'retweet_count', 'source'])

 Number of english tweets:
2305856


In [ ]:
# Train word2vec on cleaned_sentences
print("example text:")
for twt in twts:
    print(twt['text'])
    break

print("example sentence:")
for sentence in cleaned_sentences:
    print(sentence)
    break

example text:
@Walgreens Flu Shot, washing hands frequently, and getting proper amt. of rest.
example sentence:
['walgreens', 'flu', 'shot', 'washing', 'hands', 'frequently', 'getting', 'proper', 'amt']


In [ ]:
# Now train
model = make_model(cleaned_sentences,
                   "word",
                  size=100, # dimension of word vecs
                  window=5, # context size
                  min_count=100, #words repeated less than this are discarded
                  workers=6 # number of threads
                  )

# print word to vec results over all tweets for words related to 'hiv'
rel_wds = model.most_similar(positive=['hiv'], topn=10)
print('\n most similar word(s) to hiv (according to word2vec):')
print(rel_wds)

In [ ]:
# print top 30 word frequencies (counts) over all tweets
print("\n Word Frequency:")
word_dict, words, counts = get_word_freq(cleaned_sentences)
for i in range(30):
    print(words[i], counts[i])

In [ ]:
# plot all the tweets
print("\n Number of tweets with coordinates:")
print(len(coord_twts))
plot_map(coord_twts, title='plot all the tweets')

# Time series plotting:

In [ ]:
# display and plot
count_word_week = count_related_words_normalized(rel_wds, "twt_data/")
display(count_word_week)
count_word_week.plot()

# clustering of words, using word2vec distance measure (cosine similarity)
(using word2vec vectors)

In [ ]:
# make heatmap (distance between top 10 word-vectors related to 'hiv'
# over all tweets)
make_heatmap_w2vrelated(model, rel_wds=rel_wds)

# Do spectral clustering of word-vecs

In [ ]:
# PCA (visualization of all words in 2D)
X, explained_variances = scikit_pca(model, cluster="kmeans")
print("explained variance ratio: ")
print(explained_variances)

# Make histogram of word-vec similarity pairs

In [ ]:
make_histogram(X)

# ok, now lets look at phrases

In [ ]:
model = make_model(cleaned_sentences,
                   word_type = "phrase"
                  size=100, # dimension of word vecs
                  window=5, # context size
                  min_count=100, #words repeated less than this are discarded
                  workers=6 # number of threads
                  )